# Dataset Preparation for Fine-Tuning

In [1]:
import os
import random
import pandas as pd

### 1. Load a Retail Bank Intent Recognition Dataset 

+ The dataset is from huggingface https://huggingface.co/datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset 
+ 26 intents assigned to 9 categories, 25545 question/answer pairs, with approximately 1000 per intent

In [2]:
remote_file_path = "hf://datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset/bitext-retail-banking-llm-chatbot-training-dataset.parquet"
local_cache_path = "datasets/bitext-retail-banking-llm-chatbot-training-dataset.parquet"

In [3]:
# Check whether the dataset exists
if os.path.exists(local_cache_path):
    print("Loading data from local cache...")
    df_bank = pd.read_parquet(local_cache_path)
else:
    print("Downloading data from remote and caching locally...")
    df_bank = pd.read_parquet(remote_file_path)
    df_bank.to_parquet(local_cache_path, index=False)

Loading data from local cache...


In [4]:
df_bank

,tags,instruction,category,intent,response
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...
...,...,...,...,...,...
25540,BCEQZ,I'm trying to find apassword i need help,PASSWORD,set_up_password,I'm here to assist you in finding and setting ...
25541,BCIPQ,im looking for a password will uhelp me,PASSWORD,set_up_password,I'm here to assist you in setting up a passwor...
25542,BCIPZ,"I'd like to set up a password, will you help me?",PASSWORD,set_up_password,I'm here to assist you in setting up your pass...
25543,BCQ,wanna set up a password i need assistance,PASSWORD,set_up_password,I'm here to assist you in setting up your pass...


In [8]:
sampled_df_bank = df_bank.groupby('category', group_keys=False).apply(lambda x: x.sample(n=800, random_state=42))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_22172\506095088.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df_bank = df_bank.groupby('category', group_keys=False).apply(lambda x: x.sample(n=800, random_state=42))


In [9]:
sampled_df_bank = sampled_df_bank[['instruction', 'category']]
sampled_df_bank

,instruction,category
16242,i dont have a user account can ya help me open it,ACCOUNT
14964,"I'd like to close a user account, where to do it?",ACCOUNT
14197,I'm trying to find informayion about the curre...,ACCOUNT
14913,i dotn wanna keep my fucking account help me c...,ACCOUNT
15379,i got to close a fucking user account how to d...,ACCOUNT
...,...,...
8343,i have made a mistake and i have transferred m...,TRANSFER
23105,"I'd like to make a bank transfer, could I get ...",TRANSFER
8071,I'd like to cancel a fucking bank transfer he...,TRANSFER
8092,wanna cancel a fucking bank transfer to a contact,TRANSFER


In [10]:
sampled_df_bank['category'] = 'Bank'
sampled_df_bank

,instruction,category
16242,i dont have a user account can ya help me open it,Bank
14964,"I'd like to close a user account, where to do it?",Bank
14197,I'm trying to find informayion about the curre...,Bank
14913,i dotn wanna keep my fucking account help me c...,Bank
15379,i got to close a fucking user account how to d...,Bank
...,...,...
8343,i have made a mistake and i have transferred m...,Bank
23105,"I'd like to make a bank transfer, could I get ...",Bank
8071,I'd like to cancel a fucking bank transfer he...,Bank
8092,wanna cancel a fucking bank transfer to a contact,Bank


### 2. Generate a simulated dataset for non-bank inquires

In [11]:
# Define general non-bank inquiries
actions = [
    "set up a new email account", "book a hotel",
    "install a software program", "book a flight ticket", "register for an online course",
    "file my taxes", "update my profile information", "cancel a subscription",
    "connect to a Wi-Fi network", "find a lost item", "change my phone plan",
    "apply for a job", "schedule a meeting", "upgrade my device",
    "create a website", "learn a new language", "fix a technical issue",
    "pay my bills online", "track my order", "download a file"
]

subjects = [
    "Nonbank"
]

In [14]:
# Randomly generate 100
inquiries = []
for _ in range(8000):
    action = random.choice(actions)
    subject = random.choice(subjects)
    inquiry = f"I have to {action} {subject}, how can I do it?"
    inquiries.append(inquiry)

In [15]:
df_nonbank = pd.DataFrame(inquiries, columns=["instruction"])
df_nonbank['category'] = "Nonbank"
df_nonbank

,instruction,category
0,"I have to schedule a meeting Nonbank, how can ...",Nonbank
1,"I have to book a flight ticket Nonbank, how ca...",Nonbank
2,"I have to find a lost item Nonbank, how can I ...",Nonbank
3,"I have to file my taxes Nonbank, how can I do it?",Nonbank
4,"I have to connect to a Wi-Fi network Nonbank, ...",Nonbank
...,...,...
7995,"I have to book a hotel Nonbank, how can I do it?",Nonbank
7996,"I have to download a file Nonbank, how can I d...",Nonbank
7997,"I have to file my taxes Nonbank, how can I do it?",Nonbank
7998,"I have to book a hotel Nonbank, how can I do it?",Nonbank


### 3. Merge two datasets together

In [16]:
dataset = pd.concat([sampled_df_bank, df_nonbank], ignore_index=True)
dataset

,instruction,category
0,i dont have a user account can ya help me open it,Bank
1,"I'd like to close a user account, where to do it?",Bank
2,I'm trying to find informayion about the curre...,Bank
3,i dotn wanna keep my fucking account help me c...,Bank
4,i got to close a fucking user account how to d...,Bank
...,...,...
15195,"I have to book a hotel Nonbank, how can I do it?",Nonbank
15196,"I have to download a file Nonbank, how can I d...",Nonbank
15197,"I have to file my taxes Nonbank, how can I do it?",Nonbank
15198,"I have to book a hotel Nonbank, how can I do it?",Nonbank


In [17]:
# save
output_file = "datasets/generated_inquiries.csv"
dataset.to_csv(output_file, index=False)
print(f"\nInquiries saved to '{output_file}'")


Inquiries saved to 'datasets/generated_inquiries.csv'
